<a href="https://colab.research.google.com/github/nspll/Final_project/blob/master/mix_article_answer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pythainlp
!pip install gensim

    100% |████████████████████████████████| 10.3MB 3.1MB/s 
    100% |████████████████████████████████| 276kB 26.5MB/s 
  Stored in directory: /root/.cache/pip/wheels/45/24/79/022624fc914f0e559fe8a1141aaff1f9df810905a13fc75d57
Successfully built marisa-trie


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


#all methods

In [0]:
import pickle
import pandas as pd
import json
from pythainlp.tokenize import word_tokenize
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.models import Word2Vec
from pythainlp.corpus import stopwords
from pythainlp import word_vector 
from gensim.models import Word2Vec

import numpy as np

words = stopwords.words('thai')


#ARTICLE
def find_doc_article(question,model,top_n):
  '''
  หาพารากราฟที่ใช้ค้นหาคำตอบ โดยดู similarity จากโมเดลที่เทรนไว้
  question : str of untokenize question
  model : trained model using all paragraph
  all_paragraph : list of paragraphs untokenize
  return max similarity paragraph
  
  '''
  question_tokens_list =  word_tokenize(question,engine='deepcut')
  candidate_paragraph  = model.docvecs.most_similar(positive=[model.infer_vector(question_tokens_list)],topn=top_n)
  #print ("คำถาม : ",question)
  #print ("พารากราฟ : ",all_paragraph[candidate_paragraph[0][0]])
  return candidate_paragraph #article 

def demo_qa1(question,article_list):
  '''
  w2v_model คำถาม กับ w2v_model ของ article list
  '''
  #tokenize question
  question_token_list = word_tokenize(question,engine='deepcut')
  question_model =  Word2Vec([question_token_list],min_count=1)
  article_model =   Word2Vec([article_list],min_count=1)
  #find score cosine similarity of each article in article_list with all question tokens [vector]xvector
  article_score = []
  #print (question_model.wv.vocab)
  for article in article_list:
    score = 0
    for question_token in question_token_list:
        score += np.dot(article_model[article], question_model[question_token])
    article_score.append((score,article))
   #sort score
  return sorted(article_score,key=lambda x: x[0],reverse = True)[:]

def get_top_n_string_matching(str1,str_list,top_n):
  '''
  finding article with string matching
  (str,list of tupple,int) => list of dictionary
  return top_n from str_list which have most similarity with str1
  params:
  str1 : plain string , 'เฮโจเกียวอยู่ประเทศอะไร'
  str_list : tuple of id and name, [('ชิงชัน',110),('เฮโจเกียว',1203),...]
  return:
  sorted score dic {'score','article','article_id'}
  '''
  score = []
  for i in str_list:
    score.append({'score':levenshtein_ratio_and_distance(str1, i[0], ratio_calc = True),'article':i[0],'article_id':i[1]})
  return sorted(score,key=lambda x: x['score'],reverse = True)[:top_n]

def levenshtein_ratio_and_distance(s, t, ratio_calc = False):
    """ levenshtein_ratio_and_distance:
        Calculates levenshtein distance between two strings.
        If ratio_calc = True, the function computes the
        levenshtein distance ratio of similarity between two strings
        For all i and j, distance[i,j] will contain the Levenshtein
        distance between the first i characters of s and the
        first j characters of t
    """
    # Initialize matrix of zeros
    rows = len(s)+1
    cols = len(t)+1
    distance = np.zeros((rows,cols),dtype = int)

    # Populate matrix of zeros with the indeces of each character of both strings
    for i in range(1, rows):
        for k in range(1,cols):
            distance[i][0] = i
            distance[0][k] = k

    # Iterate over the matrix to compute the cost of deletions,insertions and/or substitutions    
    for col in range(1, cols):
        for row in range(1, rows):
            if s[row-1] == t[col-1]:
                cost = 0 # If the characters are the same in the two strings in a given position [i,j] then the cost is 0
            else:
                # In order to align the results with those of the Python Levenshtein package, if we choose to calculate the ratio
                # the cost of a substitution is 2. If we calculate just distance, then the cost of a substitution is 1.
                if ratio_calc == True:
                    cost = 2
                else:
                    cost = 1
            distance[row][col] = min(distance[row-1][col] + 1,      # Cost of deletions
                                 distance[row][col-1] + 1,          # Cost of insertions
                                 distance[row-1][col-1] + cost)     # Cost of substitutions
    if ratio_calc == True:
        # Computation of the Levenshtein Distance Ratio
        Ratio = ((len(s)+len(t)) - distance[row][col]) / (len(s)+len(t))
        return Ratio
    else:
        # print(distance) # Uncomment if you want to see the matrix showing how the algorithm computes the cost of deletions,
        # insertions and/or substitutions
        # This is the minimum number of edits needed to convert string a to string b
        return "The strings are {} edits away".format(distance[row][col])
      
#ANSWER
def qa_fruit_demo(question,paragraph,set_d_nb):
  '''
  (str,str,int) => dict
  finding answer with cosine similarity of w2v[question] and w2v[doc]
  params:
  question : string of question
  #paragraph : string of paragraph content of each article
  paragraph : list of words ex. ['i','love','dogs']
  set_d_nb : window size
  return dict of prop answers 
  '''
  q_token = word_tokenize(question,engine= "deepcut") #question_token = ['เฮโจเกียว','หรือ','เฮเซเกียว','เป็น','เมือง','หลวง','ของ','ประเทศ','ใด']
  p_token = paragraph#word_tokenize(paragraph,engine= "deepcut")
 
  all_pos = keyword_n_position(p_token , q_token)
  p_model = Word2Vec([p_token],min_count=1)
  q_model = Word2Vec([q_token],min_count=1)
  np_pos = []
  
  for i in all_pos[1]:
 
    for d_nb in range(set_d_nb):
        np_pos.append(i+d_nb)
        np_pos.append(i-d_nb)

  my_np_pos =  [pos for pos in np_pos if pos>=0 and pos<len(p_token)]
  p = np.array(all_pos[1])
  treshold = p.sum()/len(p)+ p.std()
  relavent_score_list = []
  #relavent = [(candidate,score)]
  for candidate in list(set(my_np_pos)): #position in paragraph
    
      rs = 0 
      #print ("candidate" ,candidate)
      for neighbor in all_pos[1]:
        if neighbor < treshold :
            #ค่า cosine similarity ของคำที่น่าจะเป็นคำตอบในพารากราฟเทียบกับทุกคำในคำถาม
            rs = cosine_relevant_score(p_token[candidate],q_token,p_model,q_model)
      relavent_score_list.append((candidate,rs))
  #print ("find_max",find_max(relavent_score)) #change to find most similarity
  if len(relavent_score_list) == 0:
    return {'score':-1}
  answer = p_token[find_max(relavent_score_list)]
  choice_answer = []
 
  relavent_score_sorted = sorted(relavent_score_list,key=lambda x: x[1],reverse = True)[:]
  #print ("find_max",find_max(relavent_score_list),"answer",p_token[find_max(relavent_score_list)])
  #print( relavent_score_sorted)
  for a in relavent_score_sorted:
     choice_answer.append(p_token[a[0]])
  
  #print ("choice_answer= ",choice_answer)
  #print ("cut answer = ",cut_ans(choice_answer,q_token))
  return { 'score':relavent_score_sorted[0][1],'choice_answer':set(cut_ans(choice_answer,q_token)),'candidate':list(set(my_np_pos)),'neighbor': all_pos[1],'neighbor_w_words': all_pos[0]}  

def qa_fruit1(question,paragraph):
  '''
  finding answer using position of answer
  params:
  question : string 
  paragraph : list of words paragraph #use to be only string
  
  '''
  question_token = word_tokenize(question,engine= "deepcut") #question_token = ['เฮโจเกียว','หรือ','เฮเซเกียว','เป็น','เมือง','หลวง','ของ','ประเทศ','ใด']
  paragraph_token = paragraph #word_tokenize(paragraph,engine= "deepcut")
  p_token = [w for w  in paragraph_token if w not in words and w not in ' ()|\?,:;/'] #คำ keyword ของพารากราฟ 
  q_token = [w for w  in question_token if w not in words and w not in ' ()|\?,:;/'] #คำ keyword ของคำถาม
  all_pos = keyword_n_position(p_token , question_token)
  np_pos = []
  set_d_nb = 3
  for i in all_pos[1]:
 
    for d_nb in range(set_d_nb):
        np_pos.append(i+d_nb)
        np_pos.append(i-d_nb)

  my_np_pos =  [pos for pos in np_pos if pos>0 and pos<len(p_token)]
  p = np.array(my_np_pos)
  treshold = p.sum()/len(p)+ p.std()
  print('threshol',treshold,all_pos[1])
  relavent_score = []
  for candidate in list(set(my_np_pos)):
    print('candidate',candidate,'tre',treshold)
    if candidate <= treshold :
      rs = 0 
      print ("candidate" ,candidate)
      for neighbor in all_pos[1]:
        if neighbor < treshold :
          tmp = candidate-neighbor
          if tmp <0 :
              tmp= -tmp
          rs = rs + tmp
      relavent_score.append((candidate,rs))
  print (relavent_score)
  if len(relavent_score) == 0 :
    return [all_pos,cut_ans,relavent_score,[],[]]
  print ("find_max",find_max(relavent_score))
  answer = p_token[find_max(relavent_score)- 3:find_max(relavent_score)+3]
  print (relavent_score)
  return [all_pos,cut_ans,relavent_score,find_max,answer,threshold]


def keyword_n_position(p_token,q_token):
  '''
  (list,list) => (dictionary,)
  return tuple of
  '''
  nb = {}
  tk_list = []
  pos_list = []
  i = 0
  for tkn in p_token:
    if tkn in q_token and tkn not in tk_list:
      if i == 0:
        print (tkn,i)
      tk_list.append(tkn)
      pos_list.append(i)
      nb[tkn] = [i]
    elif tkn in q_token and tkn in  tk_list:
      nb[tkn].append(i)
      pos_list.append(i)
    i = i + 1 
  return (nb,pos_list)

def find_max(rs):
  m_max = 0
  i_max = 0
  for i in rs:
    if i[1] >= m_max:
      i_max = i
      m_max = i[1]
  return i_max[0]

def cut_ans(ans_tkn,q_token):
  return [i for i in ans_tkn if i not in q_token and i not in ' "()\/.,' and  i not in words]

def cosine_relevant_score(p_token,q_token_list,pmodel,qmodel):
  cosine_similarity = 0
  for q_token in q_token_list:
    cosine_similarity += np.dot(pmodel[p_token], qmodel[q_token])
  return cosine_similarity 


read data

In [0]:
import pickle
#เอกสารทั้ง 4992 ชุด ที่สามารถใช้ตอบคำถามได้ ไว้สำหรับเปิดค้นหาคำตอบ
p_4992 = pickle.load( open( "/content/drive/Team Drives/pre-snporject/data/nsc_random_4892_w_100_test_file.p", "rb" ) )
p_4992_dic = pickle.load(open("/content/drive/Team Drives/pre-snporject/data/nsc_random_4892_w_100_test_file_dic_ver.p","rb"))
#ชุดคำถามคำตอบ รวมถึงเอกสารของคำตอบ 
n_100 = pickle.load(open( "/content/drive/Team Drives/pre-snporject/data/nsc_100_latest3.p","rb"))
#model doc2vec ของ เอกสารทั้งหมด 4992 เอกสาร มี 100 เอกสารทดสอบรวมอยู่ด้วยที่เหลือมาจากเอกสารสุ่ม p_5100 ตามโค้ดด้านล่าง
d2v_model = Doc2Vec.load('/content/drive/Team Drives/pre-snporject/result/test_bvg_haha.model') #4992 doc include 100 answer doc 
#model word2vec ของ เอกสารทั้งหมด 5098 เอกสาร เอาทุกเอกสารมาต่อกันฟีดทีละคำ ไม่แยกเอกสาร ใช้ 'words_list_th'
w2v_model = Word2Vec.load("/content/drive/Team Drives/pre-snporject/result/w2v_5098files_w_nsc.model")


In [16]:
# import pickle
# #เอกสารทั้ง 4998 ชุด ที่สามารถใช้ตอบคำถามได้ ไว้สำหรับเปิดค้นหาคำตอบ
# p_5100 = pickle.load( open( "/content/drive/Team Drives/pre-snporject/data/nsc_random_5000_w_100_test_file_latest_dic_version.p", "rb" ) )
# #ชุดคำถามคำตอบ รวมถึงเอกสารของคำตอบ 
# n_100 = pickle.load(open( "/content/drive/Team Drives/pre-snporject/data/nsc_100_latest3.p","rb"))
# #model doc2vec ของ เอกสารทั้งหมด 5098 เอกสาร
# #d2v_model_2 = Doc2Vec.load("/content/drive/Team Drives/pre-snporject/result/d2v_5098files_w_nsc_latest_1.model")
# d2v_model_3 = Doc2Vec.load('/content/drive/Team Drives/pre-snporject/result/test_bvg_haha.model') #4992 doc include 100 answer doc 
# #model word2vec ของ เอกสารทั้งหมด 5098 เอกสาร เอาทุกเอกสารมาต่อกันฟีดทีละคำ ไม่แยกเอกสาร ใช้ 'words_list_th'
# w2v_model = Word2Vec.load("/content/drive/Team Drives/pre-snporject/result/w2v_5098files_w_nsc.model")
# #all searching doc
# my_data = {}
# for i in n_100:
#   my_data[str(i['article_id'])] = i
# my_data.(p_5100)
# #doc_list = [a[0] for a in find_doc_article(n_100[-1]['question'],d2v_model_3,100)]
# d2v_id=set(d2v_model_3.docvecs.doctags.keys())
# len(d2v_id)
# len(p_5100)
# all_doc_id = p_5100.keys()
# all_doc_model_id = d2v_id
# all_answer_doc_id = [str(i['article_id']) for i in n_100]
# a = set(all_doc_id)
# b= set(all_doc_model_id)
# len(a.intersection(b))
# print('total doc in mix data from nsc100answer and train:',len(my_data.keys()))
# print ('total doc in train :',len(all_doc_id))
# print('total doc in model :',len(all_doc_model_id))
# print('total doc in answer :',len(all_answer_doc_id))
# print ('total doc in answer doc and in model :' ,len(set(all_answer_doc_id).intersection(all_doc_model_id)) )
# print ('total doc in train but not in model :',len(set(all_doc_id)-set(all_doc_model_id)))
# print('totlal doc in train and in answer doc :',len(set(all_doc_id).intersection(set(all_answer_doc_id))))
# print('total doc in train ,model and answer :',len(set(all_doc_id).intersection(set(all_doc_model_id).intersection(all_answer_doc_id))) )
# print('total doc in train and in model:',len(set(all_doc_id).intersection(all_doc_model_id)))

# print('total doc in mix and in model',len (set(my_data.keys()).intersection(set(all_doc_model_id))))
# print ('total doc in mix and in train',len(set(my_data.keys()).intersection(set(all_doc_id))))
# print ('total doc in mix and intra')
# # total_data = set(my_data.keys()).intersection(set())
# # cut_keys = set(my_data.keys())-set(all_doc_model_id) 
# # #len(cut_keys)
# # #for ke in cut_keys:


# #  p_5100.pop(ke)
# cut_train_doc = []
# #คัดแยกเฉพาะข้อมุลของเทรนที่มีอยู่ในโมเดลมาใช้
# for i in p_5100:
#   if i in all_doc_model_id:
#     cut_train_doc.append(p_5100[i])
# print (len(cut_train_doc))

# adding_nsc = []
# for i in n_100:
#   if str(i['article_id']) not in all_doc_id:
#     cut_train_doc.append(i)

# print(len(cut_train_doc))
# c = [str(c['article_id']) for c in cut_train_doc]
# len(set(c).intersection(set(all_doc_model_id).intersection(set(all_answer_doc_id))))
# cut_train_doc[-1]
# c_di = {}
# for i  in cut_train_doc:
#   c_di[str(i['article_id'])] = i
# print (len(c_di))


total doc in mix data from nsc100answer and train: 5191
total doc in train : 5098
total doc in model : 4992
total doc in answer : 100
total doc in answer doc and in model : 100
total doc in train but not in model : 200
totlal doc in train and in answer doc : 6
total doc in train ,model and answer : 6
total doc in train and in model: 4898
total doc in mix and in model 4992
total doc in mix and in train 5097
total doc in mix and intra


#doc2vec x position ans

In [0]:
#question = "เมืองหลวงโบราณของประเทศญี่ปุ่นคือเมืองใด"
#paragraph = "เฮโจเกียว หรือ เฮเซเกียว (ญี่ปุ่น: 平城京) เป็นเมืองหลวงโบราณของประเทศญี่ปุ่น อีกนัยหนึ่งก็คือเมืองหลวงในยุคนาระนั่นเอง ตั้งอยู่ใน จังหวัดนาระ ประเทศญี่ปุ่น มีรูปทรงเป็นแบบเวียง คือรูปทรงสี่เหลี่ยมพื้นผ้าตามแบบจีน หรือตามแบบที่นิยมสร้างกันในอาณาจักรล้านนาและพม่า ซึ่งเฮโจเกียวนี้ได้นำแบบแผนมาจากเมืองซีอานประเทศจีน เมืองหลวงของจีนในขณะนั้น"
def qa_fruit(question,paragraph):
  
  question_token = word_tokenize(question,engine= "deepcut") #question_token = ['เฮโจเกียว','หรือ','เฮเซเกียว','เป็น','เมือง','หลวง','ของ','ประเทศ','ใด']
  paragraph_token = paragraph#word_tokenize(paragraph,engine= "deepcut")
  p_token = [w for w  in paragraph_token if w not in words and w not in ' ()|\?,:;/'] #คำ keyword ของพารากราฟ 
  q_token = [w for w  in question_token if w not in words and w not in ' ()|\?,:;/'] #คำ keyword ของคำถาม
  all_pos = keyword_n_position(p_token , question_token)
  np_pos = []
  set_d_nb = 3
  for i in all_pos[1]:
 
    for d_nb in range(set_d_nb):
        np_pos.append(i+d_nb)
        np_pos.append(i-d_nb)

  my_np_pos =  [pos for pos in np_pos if pos>0 and pos<len(p_token)]
  #check bug
#   print ('p_tpken',p_token)
#   print ('q_token',q_token)
#   print ('all_pos',all_pos)
#   print('set_d_nb',set_d_nb)
#   print('my_np_pos',my_np_pos)
#   print ('np_pos',np_pos)
  #p = np.array(all_pos[1])
  p = np.array(my_np_pos)
  treshold = p.sum()/len(p)+ p.std()
#   print('threshol',treshold,all_pos[1])
  relavent_score = []
  for candidate in list(set(my_np_pos)):
#     print('candidate',candidate,'tre',treshold)
    if candidate <= treshold :
      rs = 0 
#       print ("candidate" ,candidate)
      for neighbor in all_pos[1]:
        if neighbor < treshold :
          tmp = candidate-neighbor
          if tmp <0 :
              tmp= -tmp
          rs = rs + tmp
          #rint("neighbor = ",neighbor,"tmp = ", tmp ,"rs = ",rs)
      relavent_score.append((candidate,rs))
#   print (relavent_score)
  if len(relavent_score) == 0:
    return -1
#   print ("find_max",find_max(relavent_score))
  answer = p_token[find_max(relavent_score)- 3:find_max(relavent_score)+3]
#   print (relavent_score)






#Sol1 cut answerคือrelevant scoreที่เอาคำรอบข้าง(+-3)
#   print (find_max(relavent_score))
#   print ("answer= ",answer)
  choice_answer = []
#   print ("cut answer = ",cut_ans(answer,q_token))
  return relavent_score[0][1]

def keyword_n_position(p_token,question_token):
  nb = {}
  tk_list = []
  pos_list = []
  i = 0
  for tkn in p_token:
    if tkn in question_token and tkn not in tk_list:
      tk_list.append(tkn)
      pos_list.append(i)
      nb[tkn] = [i]
    elif tkn in question_token and tkn in  tk_list:
      nb[tkn].append(i)
      pos_list.append(i)
    i = i + 1 
  return (nb,pos_list)

def find_max(rs):
  if len(rs) == 0:
    return ''
  m_max = 0
  i_max = 0
  for i in rs:
    if i[1] >= m_max:
      i_max = i
      m_max = i[1]
  return i_max[0]

def cut_ans(ans_tkn,q_token):
  return [i for i in ans_tkn if i not in q_token]

ตรวจสอบคุณสมบัติ doc2vec model ที่เทรนมา bvg_haha.model
จำนวน doc และข้อมูลข้างใน 

In [0]:
qlt_model = Doc2Vec.load('/content/drive/Team Drives/pre-snporject/result/test_bvg_haha.model') #4992 doc include 100 answer doc 
#qlt_model.delete_temporary_training_data(keep_doctags_vectors=True, keep_inference=True)

In [54]:
cnt_have_answer = 0
cnt_have_correct_article = 0
for i in n_100:
  question = i['question']
  doc_id_list =[d[0] for d in find_doc_article(question,d2v_model,100)] #คัดเลือกหัวข้อที่เกี่ยวข้องด้วย doc2vec model กับ infer vector คำถาม
  max_score = 0
  answer_doc_id = ''
  print("==============================================================================")
  print(question,'หัวข้อที่เกี่ยวข้องทั้งหมด',doc_id_list,len(set(doc_id_list)))
  if len(doc_id_list) != 0:
    #ค้นหาคำตอบของแต่ละหัวข้อ
    for cnt,doc_id in enumerate(doc_id_list):
          answer_score = qa_fruit(question,p_4992_dic[str(doc_id)]['words_list_th'])
          
          if answer_score == -1:
            b.append((cnt ,p_4992_dic[str(doc_id)]['article'],'no answer'))
            
          elif max_score <= answer_score:
              max_score = answer_score
              answer_doc_id = doc_id
              #print('yes',question,answer_doc_id,p_4992_dic[str(answer_doc_id)]['article'])
    if max_score == 0:
        print('cannot find answer')
    else:
        print('this question have answer in article named : ',p_4992_dic[str(answer_doc_id)]['article'],'with score of position method answer',max_score)
        print('selected correct article :',str(answer_doc_id)==str(i['article_id']))
        cnt_have_answer = cnt_have_answer+1
        if str(answer_doc_id)==str(i['article_id']):
          cnt_have_correct_article += 1

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:26: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:135: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:105: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:127: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


เอเลียนส์เป็นภาพยนตร์แนวไหน หัวข้อที่เกี่ยวข้องทั้งหมด ['232225', '104336', '2205', '110157', '134851', '136850', '109993', '138774', '110122', '136830', '101661', '111918', '14844', '110150', '110793', '110876', '103892', '104055', '104016', '110675', '105411', '144328', '102366', '103900', '104408', '104052', '104012', '103918', '103906', '144326', '103907', '103929', '110147', '110671', '103916', '235740', '102364', '101671', '103903', '136836', '101660', '102370', '102365', '103925', '103926', '109935', '112625', '103912', '101665', '103922', '103899', '101672', '104412', '104042', '104044', '101662', '107836', '102367', '101677', '103909', '103914', '181806', '114436', '101958', '104393', '104155', '105094', '104075', '103923', '100852', '104405', '104019', '104050', '104392', '125094', '104020', '104853', '134490', '104400', '104141', '100854', '103904', '10172', '104013', '110305', '21852', '104398', '103931', '231921', '101668', '102359', '104404', '104395', '103921', '104158',

In [55]:
print (cnt_have_answer ,cnt_have_correct_article )

64 5


สรุปผลการค้นหาเอกสารที่เกี่ยวข้องจากคำถามด้วย

ค้นหาเอกสาร : doc2vec ,top_n = 100

ค้นหาคำตอบ : position , set_nb = 3 ,treshold สำหรับจำกัดการค้นหาที่ไม่จำเป็น method : qa_fruit

สามารถเลือกเอกสารได้ทั้งหมด 64 ข้อ โดยยึดจากเอกสารที่มีค่าคะแนนคำตอบสูงที่สุด

โดยเลือกเอกสารได้ถูกต้องทั้งหมด 5 ข้อ


#DOC2VEC x cosinerelevant

In [16]:
cnt_have_answer = 0
cnt_have_correct_article = 0
b= []
for i in n_100:
  question = i['question']
  doc_id_list =[d[0] for d in find_doc_article(question,d2v_model,100)] #คัดเลือกหัวข้อที่เกี่ยวข้องด้วย doc2vec model กับ infer vector คำถาม
  max_score = 0
  answer_doc_id = ''
  print("==============================================================================")
  print(question,'หัวข้อที่เกี่ยวข้องทั้งหมด',doc_id_list,len(set(doc_id_list)))
  if len(doc_id_list) != 0:
    #ค้นหาคำตอบของแต่ละหัวข้อ
    for cnt,doc_id in enumerate(doc_id_list):
          answer_score =  qa_fruit_demo(question,p_4992_dic[str(doc_id)]['words_list_th'],3)['score']
          
          if answer_score == -1:
            b.append((cnt ,p_4992_dic[str(doc_id)]['article'],'no answer'))
            
          elif max_score <= answer_score:
              max_score = answer_score
              answer_doc_id = doc_id
              #print('yes',question,answer_doc_id,p_4992_dic[str(answer_doc_id)]['article'])
    if max_score == 0:
        print('cannot find answer')
    else:
        print('this question have answer in article named : ',p_4992_dic[str(answer_doc_id)]['article'],'with score of position method answer',max_score)
        print('selected correct article :',str(answer_doc_id)==str(i['article_id']))
        cnt_have_answer = cnt_have_answer+1
        if str(answer_doc_id)==str(i['article_id']):
          cnt_have_correct_article += 1

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:140: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:135: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:105: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:127: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


เอเลียนส์เป็นภาพยนตร์แนวไหน หัวข้อที่เกี่ยวข้องทั้งหมด ['232225', '104336', '110122', '144326', '110157', '2205', '136830', '109993', '138774', '136850', '110150', '110675', '110793', '134851', '10172', '136836', '123863', '134863', '110671', '104408', '103929', '103907', '184197', '110147', '105411', '101431', '104412', '104012', '125094', '144328', '104405', '197383', '104020', '235740', '104016', '110305', '101660', '102370', '101671', '103906', '101958', '104042', '103126', '102359', '103918', '102366', '103892', '111918', '206718', '103914', '101060', '104393', '104392', '104388', '101672', '102365', '191712', '104075', '104157', '103919', '105092', '105057', '112694', '194894', '104155', '104398', '103923', '101665', '104407', '138754', '104018', '121903', '104057', '201438', '103903', '103921', '103912', '140820', '100092', '104404', '103927', '105094', '107836', '103909', '103916', '109659', '103922', '104158', '103899', '104055', '14844', '101432', '102363', '166458', '104395'

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:252: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


this question have answer in article named :  กันกันวิง with score of position method answer 0.0012493008925957838
selected correct article : False
พระเจ้าอาฟองโซที่ 1 แห่งโปรตุเกส เป็นผู้สืบเชื้อสายจากราชวงศ์ใด หัวข้อที่เกี่ยวข้องทั้งหมด ['232225', '110150', '10172', '104336', '110122', '144326', '109993', '110675', '110147', '110793', '138774', '110671', '136836', '110157', '144328', '103903', '136830', '136850', '2205', '102366', '103926', '102370', '103929', '125094', '103915', '111918', '102361', '112694', '104042', '101671', '104050', '102365', '104046', '103925', '104040', '101660', '103901', '102364', '101665', '104021', '110305', '104020', '103921', '101661', '103920', '103900', '104055', '103919', '134851', '104053', '104135', '104060', '103918', '102367', '123863', '104043', '103909', '104016', '103914', '103927', '103907', '103916', '103923', '104013', '104052', '104014', '104039', '103931', '104157', '102359', '104012', '104018', '105092', '104056', '103899', '101672', '10

In [17]:
print (cnt_have_answer ,cnt_have_correct_article )

90 7


สรุปผลการค้นหาเอกสารที่เกี่ยวข้องจากคำถามด้วย

ค้นหาเอกสาร : doc2vec ,top_n = 100

ค้นหาคำตอบ : cosine relevant , set_nb = 3 ,treshold สำหรับจำกัดการค้นหาที่ไม่จำเป็น method : qa_fruit_demo

สามารถเลือกเอกสารได้ทั้งหมด 90  ข้อ โดยยึดจากเอกสารที่มีค่าคะแนนคำตอบสูงที่สุด

โดยเลือกเอกสารได้ถูกต้องทั้งหมด  7 ข้อ


#DOC2VEC x cosinerelevant set_nb = 10

#DOC2VEC x string matching

In [25]:
doc_id_list =[d[0] for d in find_doc_article('แม่น้ำใดไหลตัดผ่านเทือกเขาตะนาวศรี',d2v_model,100)] 
str_list = [ (p_4992_dic[str(i)]['article'],i) for i in doc_id_list]
print(str_list)
get_top_n_string_matching(question,str_list,10)

[('พระยาจุฬาราชมนตรี (สน)', '108042'), ('แม่น้ำอามูดาร์ยา', '110509'), ('แม่น้ำยมุนา', '110010'), ('แม่น้ำแควใหญ่', '201295'), ('อุทยานแห่งชาติพุเตย', '140546'), ('สมการกำลังสอง', '116292'), ('พระยาจุฬาราชมนตรี (แก้ว)', '108044'), ('เสนาสนะ', '107939'), ('แม่น้ำโก-ลก', '180464'), ('วิปัสสนาธุระ', '111873'), ('สรีดภงส์', '209494'), ('ค่าคงตัวออยเลอร์-แมสเชโรนี', '102358'), ('พระยายมราช (หมัด)', '110970'), ('พระยาจุฬาราชมนตรี (น้อย)', '110317'), ('การปลูกหม่อนเลี้ยงไหม', '238505'), ('พระยาจุฬาราชมนตรี (เซน)', '111268'), ('เทือกเขาตะนาวศรี', '373102'), ('ทะเลอาหรับ', '101715'), ('อักษรลิเชีย', '101966'), ('Specular reflection', '217930'), ('อำเภอวังทรายพูน', '123415'), ('จังหวัดนาทวี', '143386'), ('การหมุนควง', '122888'), ('สมเด็จพระสันตะปาปาเฟลิกซ์ที่ 4', '104021'), ('เมทริกซ์แบบบล็อก', '136630'), ('ปัพพาชนียกรรม', '111022'), ('ทรงสิบสองหน้าปลายตัดเมทาไบออกเมนต์', '103281'), ('กระดูกไฮออยด์', '168037'), ('ภาษาสวาน', '123707'), ('เทือกเขา', '168387'), ('ภาวะหัวใจวาย', '165707'), ('ทรงลูกบ

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[{'article': 'เอเชียตะวันออกเฉียงใต้',
  'article_id': '2074',
  'score': 0.3673469387755102},
 {'article': 'สยามเซ็นเตอร์', 'article_id': '102112', 'score': 0.35},
 {'article': 'เทือกเขาตะนาวศรี',
  'article_id': '373102',
  'score': 0.32558139534883723},
 {'article': 'เนื้อเยื่อกล้ามเนื้อลาย', 'article_id': '104484', 'score': 0.32},
 {'article': 'เครื่องปฏิกรณ์นิวเคลียร์',
  'article_id': '109379',
  'score': 0.3137254901960784},
 {'article': 'การพิสูจน์ว่า e เป็นจำนวนอตรรกยะ',
  'article_id': '10550',
  'score': 0.3050847457627119},
 {'article': 'ทรงลูกบาศก์ปลายตัดออกเมนต์',
  'article_id': '103031',
  'score': 0.3018867924528302},
 {'article': 'กฎข้อที่สามของอุณหพลศาสตร์',
  'article_id': '144406',
  'score': 0.3018867924528302},
 {'article': 'แหลมเทียนหยาไหเจี่ยว',
  'article_id': '107570',
  'score': 0.2978723404255319},
 {'article': 'พระที่นั่งจักรวรรดิ์ไพชยนต์',
  'article_id': '102686',
  'score': 0.2962962962962963}]

In [0]:
 get_top_n_string_matching(question,str_list,0)[0]['score']

In [27]:
cnt_have_answer = 0
cnt_have_correct_article = 0
b= []
for i in n_100:
  question = i['question']
  doc_id_list =[d[0] for d in find_doc_article(question,d2v_model,100)] #คัดเลือกหัวข้อที่เกี่ยวข้องด้วย doc2vec model กับ infer vector คำถาม
  max_score = 0
  answer_doc_id = ''
  print("==============================================================================")
  print(question,'หัวข้อที่เกี่ยวข้องทั้งหมด',doc_id_list,len(set(doc_id_list)))
  str_list = [ (p_4992_dic[str(i)]['article'],i) for i in doc_id_list]
  if len(doc_id_list) != 0:
    #ค้นหาคำตอบของแต่ละหัวข้อ
    for cnt,doc_id in enumerate(doc_id_list):
      
          answer_score =  get_top_n_string_matching(question,str_list,1)[0]['score']
          
          if answer_score == -1:
            b.append((cnt ,p_4992_dic[str(doc_id)]['article'],'no answer'))
            
          elif max_score <= answer_score:
              max_score = answer_score
              answer_doc_id = doc_id
              #print('yes',question,answer_doc_id,p_4992_dic[str(answer_doc_id)]['article'])
    if max_score == 0:
        print('cannot find answer')
    else:
        print('this question have answer in article named : ',p_4992_dic[str(answer_doc_id)]['article'],'with score of position method answer',max_score)
        print('selected correct article :',str(answer_doc_id)==str(i['article_id']))
        cnt_have_answer = cnt_have_answer+1
        if str(answer_doc_id)==str(i['article_id']):
          cnt_have_correct_article += 1

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


เอเลียนส์เป็นภาพยนตร์แนวไหน หัวข้อที่เกี่ยวข้องทั้งหมด ['2205', '104336', '103918', '110122', '103929', '103907', '102366', '110150', '232225', '144328', '101661', '103909', '104055', '104050', '109993', '110675', '101671', '197383', '105411', '101668', '103923', '103927', '184197', '110793', '104012', '104016', '102370', '111918', '102364', '123863', '104053', '103891', '110157', '14844', '104155', '103900', '103903', '101665', '104060', '102365', '103921', '101660', '104020', '103892', '144326', '101676', '104015', '103916', '104044', '103906', '103904', '101659', '103925', '103919', '110671', '103899', '103915', '104021', '103901', '103914', '103912', '104052', '104061', '103926', '102367', '101662', '104043', '104018', '181806', '102361', '104013', '103920', '101670', '104040', '134851', '101672', '107836', '104042', '219409', '103922', '110147', '125094', '101675', '104129', '104039', '104046', '104057', '104130', '104139', '134490', '104019', '136850', '110313', '104158', '104045

In [28]:
print (cnt_have_answer ,cnt_have_correct_article )

100 0


ไม่ได้ช่วยอะไร

In [13]:
len(d2v_model_3.docvecs.doctags.keys())

4898

# String mathcing x position

In [0]:
#question = "เมืองหลวงโบราณของประเทศญี่ปุ่นคือเมืองใด"
#paragraph = "เฮโจเกียว หรือ เฮเซเกียว (ญี่ปุ่น: 平城京) เป็นเมืองหลวงโบราณของประเทศญี่ปุ่น อีกนัยหนึ่งก็คือเมืองหลวงในยุคนาระนั่นเอง ตั้งอยู่ใน จังหวัดนาระ ประเทศญี่ปุ่น มีรูปทรงเป็นแบบเวียง คือรูปทรงสี่เหลี่ยมพื้นผ้าตามแบบจีน หรือตามแบบที่นิยมสร้างกันในอาณาจักรล้านนาและพม่า ซึ่งเฮโจเกียวนี้ได้นำแบบแผนมาจากเมืองซีอานประเทศจีน เมืองหลวงของจีนในขณะนั้น"
def qa_fruit(question,paragraph):
  
  question_token = word_tokenize(question,engine= "deepcut") #question_token = ['เฮโจเกียว','หรือ','เฮเซเกียว','เป็น','เมือง','หลวง','ของ','ประเทศ','ใด']
  paragraph_token = paragraph#word_tokenize(paragraph,engine= "deepcut")
  p_token = [w for w  in paragraph_token if w not in words and w not in ' ()|\?,:;/'] #คำ keyword ของพารากราฟ 
  q_token = [w for w  in question_token if w not in words and w not in ' ()|\?,:;/'] #คำ keyword ของคำถาม
  all_pos = keyword_n_position(p_token , question_token)
  np_pos = []
  set_d_nb = 3
  for i in all_pos[1]:
 
    for d_nb in range(set_d_nb):
        np_pos.append(i+d_nb)
        np_pos.append(i-d_nb)

  my_np_pos =  [pos for pos in np_pos if pos>0 and pos<len(p_token)]
  #check bug
#   print ('p_tpken',p_token)
#   print ('q_token',q_token)
#   print ('all_pos',all_pos)
#   print('set_d_nb',set_d_nb)
#   print('my_np_pos',my_np_pos)
#   print ('np_pos',np_pos)
  #p = np.array(all_pos[1])
  p = np.array(my_np_pos)
  treshold = p.sum()/len(p)+ p.std()
#   print('threshol',treshold,all_pos[1])
  relavent_score = []
  for candidate in list(set(my_np_pos)):
#     print('candidate',candidate,'tre',treshold)
    if candidate <= treshold :
      rs = 0 
#       print ("candidate" ,candidate)
      for neighbor in all_pos[1]:
        if neighbor < treshold :
          tmp = candidate-neighbor
          if tmp <0 :
              tmp= -tmp
          rs = rs + tmp
          #rint("neighbor = ",neighbor,"tmp = ", tmp ,"rs = ",rs)
      relavent_score.append((candidate,rs))
#   print (relavent_score)
  if len(relavent_score) == 0:
    return -1
#   print ("find_max",find_max(relavent_score))
  answer = p_token[find_max(relavent_score)- 3:find_max(relavent_score)+3]
#   print (relavent_score)






#Sol1 cut answerคือrelevant scoreที่เอาคำรอบข้าง(+-3)
#   print (find_max(relavent_score))
#   print ("answer= ",answer)
  choice_answer = []
#   print ("cut answer = ",cut_ans(answer,q_token))
  return relavent_score[0][1]

def keyword_n_position(p_token,question_token):
  nb = {}
  tk_list = []
  pos_list = []
  i = 0
  for tkn in p_token:
    if tkn in question_token and tkn not in tk_list:
      tk_list.append(tkn)
      pos_list.append(i)
      nb[tkn] = [i]
    elif tkn in question_token and tkn in  tk_list:
      nb[tkn].append(i)
      pos_list.append(i)
    i = i + 1 
  return (nb,pos_list)

def find_max(rs):
  if len(rs) == 0:
    return ''
  m_max = 0
  i_max = 0
  for i in rs:
    if i[1] >= m_max:
      i_max = i
      m_max = i[1]
  return i_max[0]

def cut_ans(ans_tkn,q_token):
  return [i for i in ans_tkn if i not in q_token]

In [0]:
cnt_have_answer = 0
cnt_have_correct_article = 0
str_list = [(p['article'],p['article_id']) for p in p_4992]
b= []
for i in n_100:
  question = i['question']
  
  
  doc_id_list =[d['article_id'] for d in  get_top_n_string_matching(question,str_list,100)] #คัดเลือกหัวข้อที่เกี่ยวข้องด้วย article list with question
  max_score = 0
  answer_doc_id = ''
  print("==============================================================================")
  print(question,'หัวข้อที่เกี่ยวข้องทั้งหมด',doc_id_list,len(set(doc_id_list)))
  if len(doc_id_list) != 0:
    #ค้นหาคำตอบของแต่ละหัวข้อ
    for cnt,doc_id in enumerate(doc_id_list):
          answer_score = qa_fruit(question,p_4992_dic[str(doc_id)]['words_list_th'])
          
          if answer_score == -1:
            b.append((cnt ,p_4992_dic[str(doc_id)]['article'],'no answer'))
            
          elif max_score <= answer_score:
              max_score = answer_score
              answer_doc_id = doc_id
              #print('yes',question,answer_doc_id,p_4992_dic[str(answer_doc_id)]['article'])
    if max_score == 0:
        print('cannot find answer')
    else:
        print('this question have answer in article named : ',p_4992_dic[str(answer_doc_id)]['article'],'with score of position method answer',max_score)
        print('selected correct article :',str(answer_doc_id)==str(i['article_id']))
        cnt_have_answer = cnt_have_answer+1
        if str(answer_doc_id)==str(i['article_id']):
          cnt_have_correct_article += 1

เอเลียนส์เป็นภาพยนตร์แนวไหน หัวข้อที่เกี่ยวข้องทั้งหมด ['146137', '102710', '103184', '108320', '163189', '110683', '111891', '200961', '108474', '179539', '113414', '107266', '1041', '108339', '113420', '103580', '22561', '107674', '160693', '192975', '132096', '15635', '106425', '102034', '10553', '141006', '178607', '230186', '142094', '106136', '157860', '120733', '10114', '11012', '10747', '175387', '102842', '101736', '110082', '130135', '113154', '102104', '113694', '100495', '134908', '122104', '106377', '183414', '171870', '143778', '107381', '107269', '101759', '210403', '219734', '130630', '112942', '112986', '135804', '124568', '2074', '232179', 397951, '111672', '111030', '111450', '105411', '106037', '100133', '113876', '113867', '161989', '110753', '103191', '112076', '113631', '10108', '107881', '107157', '110365', '239660', '155870', '100697', '106151', '144614', '16807', '232005', '106326', '102112', '106668', '10132', '147809', '109622', '100219', '112872', '10392', 

Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:26: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:135: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:105: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:127: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


this question have answer in article named :  รายชื่อโรงภาพยนตร์ในประเทศไทย with score of position method answer 98263
selected correct article : False
พระเจ้าอาฟองโซที่ 1 แห่งโปรตุเกส เป็นผู้สืบเชื้อสายจากราชวงศ์ใด หัวข้อที่เกี่ยวข้องทั้งหมด [289367, '155034', '172137', '237351', '154822', '13092', '235469', '237609', '158568', '235754', '235511', '153264', '157344', '219163', 524853, '159190', '101759', '218601', '178210', '233965', '218580', '110589', '113106', '218708', '151779', '151557', '113281', '180061', '218392', '110873', '110498', '105406', '172213', '111115', '111850', 857573, '112872', '110503', '101901', '101380', '239459', '114390', '111429', '115286', '153202', '10867', '10490', '100234', '102687', '113012', '158996', '10848', '215127', '225591', '100697', '111540', '189129', '105441', '104951', '110048', '114375', '110412', '113011', '191832', '159999', '218213', '110324', '217681', '228156', '102976', '230793', '227322', '166552', '103439', '142903', '106194', '12499

In [0]:
print (cnt_have_answer ,cnt_have_correct_article )

98 5


ผลจากการเลือกหัวเรื่องด้วยวิธี string matching รายชื่อหัวข้อ กับคำถาม 100 อันดับแรก และเลือกหัวข้อที่มีค่าคำตอบสูงสุด ซึ่งได้จากวิธีการหาคำตอบแบบ position

สามารถเลือกเอกสารที่หาคำตอบได้ทั้งสิ้น จำนวน 98 ข้อ

และเมื่อเลือกเอกสารที่มีค่าคำตอบสูงสุดนั้น เลือกมาได้ถูกต้อง 5 ข้อ

สรุปได้ว่า วิธีการเลือกหัวข้อแบบ string matching นั้นให้ประสิทธิภาพที่ดี 


#NEW SECTION

In [10]:

for i in n_100:
  question = i['question']
  doc_id_list =[d[0]  for d in find_doc_article(question,d2v_model_3,100)]
  
  #cdd = []
  max_score = 0
  answer_doc_id = ''
  print(i['question'])
  if len(doc_id_list) != 0 :
    for doc_id in doc_id_list:
      answer_score = qa_fruit(question,p_5100[str(doc_id)]['words_list_th'])
      print (answer_score)
      if answer_score == -1:
          print(p_5100[str(doc_id)]['article'],'no answer')
      else:    
            if max_score <= answer_score:
              max_score = answer_score
              answer_doc_id = doc_id
            print('yes',question,answer_doc_id,p_5100[str(answer_doc_id)]['article'])

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:26: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:135: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:105: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:127: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


เอเลียนส์เป็นภาพยนตร์แนวไหน
-1
NFC no answer
-1
รายชื่อวงศ์พรรณไม้ no answer
-1
พลาโคซัว no answer
-1
ประเทศสวีเดนในโอลิมปิกฤดูร้อน 2004 no answer
-1
18 กรกฎาคม no answer
-1
นาโธสโทมูลิดา no answer
-1
ไคโนรินชา no answer
-1
ประเทศสเปนในโอลิมปิกฤดูร้อน 2004 no answer
-1
อภินิหารเจ้าหนูตะลุยแดนเวทย์ no answer
-1
ไดยีมิดา no answer
-1
ออร์โธเนคทิดา no answer
-1
สมเด็จพระสันตะปาปาสตีเฟนที่ 8 no answer
-1
ลอริซิเฟอรา no answer
-1
โพโกโนฟอรา no answer
-1
โครมาลวีโอลาตา no answer
-1
ประทัดจีน no answer
-1
สมเด็จพระสันตะปาปาลันโด no answer
-1
สมเด็จพระสันตะปาปาฮิลาริอุส no answer
-1
สมเด็จพระสันตะปาปาจอห์นที่ 8 no answer
-1
สมเด็จพระสันตะปาปาจอห์นที่ 4 no answer
-1
สรุปเหรียญโอลิมปิกฤดูร้อน 1920 no answer
-1
สมเด็จพระสันตะปาปาสตีเฟนที่ 7 no answer
-1
สมเด็จพระสันตะปาปาอนิเซตุส no answer
-1
สมเด็จพระสันตะปาปาเซอร์จิอุสที่ 2 no answer
-1
ตำนานโหดโคตรไอ้เคี่ยม no answer
-1
สมเด็จพระสันตะปาปาเอเดรียนที่ 2 no answer
-1
สมเด็จพระสันตะปาปาโฮโนริอุสที่ 1 no answer
-1
สมเด็จพระสันตะปาปาบอนิเฟซที่ 2 no 

KeyError: ignored